## Модель BERT
Возьмем прдобученную на русском языке модель RuBERT. Попробуем дообучить его на задачу классификации тональности с помощью датасета [RuSentiment](https://huggingface.co/datasets/MonoHime/ru_sentiment_dataset).

In [1]:
from transformers import (
    AutoTokenizer,
    AutoModelForSequenceClassification,
    pipeline,
    DataCollatorWithPadding,
    Trainer,
    TrainingArguments,
)
from transformers import 
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix
from sklearn.model_selection import train_test_split
import datasets
import evaluate
import numpy as np
import torch
import time
from tqdm import tqdm

In [2]:
# предобученная модель
model_name = "DeepPavlov/rubert-base-cased-sentence"

# загрузка модели и токенизатора
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForSequenceClassification.from_pretrained(model_name,
    num_labels=3,                 # NEUTRAL, POSITIVE, NEGATIVE
    ignore_mismatched_sizes=True  # Игнорируем несоответствие размеров для базового тестирования
)

print(f"Модель загружена! Параметров: {model.num_parameters():,}")
print(f"Устройство: {'CUDA' if torch.cuda.is_available() else 'CPU'}")

Модель загружена! Параметров: 177,855,747
Устройство: CUDA


In [3]:
# соответствие меток
print(model.config.id2label)
print(model.config.label2id)

{0: 'LABEL_0', 1: 'LABEL_1', 2: 'LABEL_2'}
{'LABEL_0': 0, 'LABEL_1': 1, 'LABEL_2': 2}


In [4]:
model.config.id2label = {
    0: 'NEUTRAL',
    1: 'POSITIVE',
    2: 'NEGATIVE'
}
model.config.label2id = {
    'NEUTRAL':  0,
    'POSITIVE': 1,
    'NEGATIVE': 2
}
print(model.config.id2label)
print(model.config.label2id)

{0: 'NEUTRAL', 1: 'POSITIVE', 2: 'NEGATIVE'}
{'NEUTRAL': 0, 'POSITIVE': 1, 'NEGATIVE': 2}


In [5]:
pipe = pipeline(
    task="sentiment-analysis",
    model=model,
    tokenizer=tokenizer,
    device=0 if torch.cuda.is_available() else -1
)
pipe('Отличное место!')

[{'label': 'POSITIVE', 'score': 0.3956296741962433}]


In [8]:
pipe([
    'Отличное место!',
    'Спасибо деду за победу!',
    'Я не знаю, что делать',
    'Мне всегда не везет :('
])

[{'label': 'POSITIVE', 'score': 0.3956296741962433},
 {'label': 'NEGATIVE', 'score': 0.38501259684562683},
 {'label': 'NEGATIVE', 'score': 0.3968161940574646},
 {'label': 'NEGATIVE', 'score': 0.38515138626098633}]


### Дообучение на датасете
Дообучать будем только "голову" модели - верхний модуль, который отвечает за классификацию, чтобы максимально сократить количество обучаемых параметров.

In [11]:
# дообучение BERT - только classification head
print("="*50)
print("ДООБУЧЕНИЕ BERT (ТОЛЬКО ГОЛОВА)")
print("="*50)

# заморозим все слои BERT, кроме головы классификации
for name, param in model.named_parameters():
    if 'classifier' not in name:  # Замораживаем все, кроме classifier
        param.requires_grad = False
    else:
        param.requires_grad = True
        print(f"Обучаемый параметр: {name}")

trainable_params = sum(p.numel() for p in model.parameters() if p.requires_grad)
total_params = sum(p.numel() for p in model.parameters())

print(f"\nОбучаемых параметров: {trainable_params:,}")
print(f"Всего параметров: {total_params:,}")
print(f"Доля обучаемых: {trainable_params/total_params*100:.2f}%")

ДООБУЧЕНИЕ BERT (ТОЛЬКО ГОЛОВА)
Обучаемый параметр: classifier.weight
Обучаемый параметр: classifier.bias

Обучаемых параметров: 2,307
Всего параметров: 177,855,747
Доля обучаемых: 0.00%


In [12]:
# загрузка датасета RuSentiment
df = datasets.load_dataset("MonoHime/ru_sentiment_dataset")

In [13]:
df_train, df_test = df['train'].to_pandas(), df['validation'].to_pandas()
df_train, df_test = df_train.drop('Unnamed: 0', axis=1), df_test.drop('Unnamed: 0', axis=1)

In [14]:
df_train.shape, df_test.shape

((189891, 2), (21098, 2))


In [15]:
# разделение на трейн и валидацию для замеров качества в процессе обучения
df_train, df_val = train_test_split(df_train,
                                    train_size=0.8,
                                    shuffle=True,
                                    stratify=df_train['sentiment'],
                                    random_state=42)

df_train, df_val, df_test = df_train.reset_index(drop=True), df_val.reset_index(drop=True), df_test.reset_index(drop=True)

In [16]:
df_train.shape[0], df_val.shape[0], df_test.shape[0]

(151912, 37979, 21098)


In [20]:
df = datasets.DatasetDict({
    'train': datasets.Dataset.from_pandas(df_train),
    'validation': datasets.Dataset.from_pandas(df_val),
    'test': datasets.Dataset.from_pandas(df_test)
})

df

DatasetDict({
    train: Dataset({
        features: ['text', 'sentiment'],
        num_rows: 151912
    })
    validation: Dataset({
        features: ['text', 'sentiment'],
        num_rows: 37979
    })
    test: Dataset({
        features: ['text', 'sentiment'],
        num_rows: 21098
    })
})

In [21]:
df['train'][0]

{'text': 'Выражаю благодарность Выражаю огромную благодарность Леониду, Любови и Ольге Викторовне за тот уют, тепло, комфорт, который нам предоставили в хостеле "Москва 2000". Идеальная чистота, порядок, радушие хозяев, а самое главное они всегда готовы помочь во всем и помогают абсолютно всем. Огромное Вам СПАСИБО! ',
 'sentiment': 1}

In [22]:
# функция токенизации

def tokenize_function(examples):
  return tokenizer(
      examples['text'],
      truncation=True,
      max_length=model.config.max_position_embeddings,
      padding=False,  # динамический padding в collator
  )

In [23]:
tokenized_df = df.map(tokenize_function, batched=True, remove_columns=['text'])
tokenized_df = tokenized_df.rename_columns({'sentiment': 'labels'})
tokenized_df

Map:   0%|          | 0/151912 [00:00<?, ? examples/s]

Map:   0%|          | 0/37979 [00:00<?, ? examples/s]

Map:   0%|          | 0/21098 [00:00<?, ? examples/s]

DatasetDict({
    train: Dataset({
        features: ['labels', 'input_ids', 'token_type_ids', 'attention_mask'],
        num_rows: 151912
    })
    validation: Dataset({
        features: ['labels', 'input_ids', 'token_type_ids', 'attention_mask'],
        num_rows: 37979
    })
    test: Dataset({
        features: ['labels', 'input_ids', 'token_type_ids', 'attention_mask'],
        num_rows: 21098
    })
})

In [24]:
# коллатор для динамического паддинга
data_collator = DataCollatorWithPadding(tokenizer=tokenizer)

In [25]:
# устанавливаем формат для PyTorch
tokenized_df.set_format('torch', columns=['input_ids', 'attention_mask', 'labels'])

In [28]:
accuracy = evaluate.load('accuracy')
precision = evaluate.load('precision')
recall = evaluate.load('recall')
f1_score = evaluate.load('f1')

In [29]:
# функция для вычисления метрик при обучении и тестировании
def compute_metrics(eval_pred: tuple) -> dict:
  logits, labels = eval_pred
  predictions = np.argmax(logits, axis=-1)

  return {
      'accuracy': accuracy.compute(predictions, labels)['accuracy'],
      'precision_macro': precision.compute(predictions, labels, average='macro')['precision'],
      'recall_macro': recall.compute(predictions, labels, average='macro')['recall'],
      'f1_macro': f1_score.compute(predictions, labels, average='macro')['f1']
  }

In [32]:
# настройки обучения

training_args = TrainingArguments(
    output_dir='./training',
    num_train_epochs=3,               # Количество эпох
    per_device_train_batch_size=24,   # Размер батча
    per_device_eval_batch_size=48,
    warmup_ratio=0.1,                 # 10% шагов - плавное увеличение LR с 0 до learning_rate
    weight_decay=0.01,                # L2 регуляризация
    learning_rate=5e-4,
    eval_strategy='epoch',            # Оценка после каждой эпохи
    save_strategy='epoch',            # Сохранение после каждой эпохи
    load_best_model_at_end=True,      # Загрузка лучшей модели в конце
    metric_for_best_model='accuracy',
    greater_is_better=True,           # Чем больше метрика - тем лучше
    report_to='none',                 # Отключение WandD/TensorBoard
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_df["train"],
    eval_dataset=tokenized_df["validation"],
    compute_metrics=compute_metrics,
    data_collator=data_collator,
)

In [ ]:
# метрики до обучения
eval_results_before = trainer.evaluate(eval_dataset=tokenized_df['test'])

In [ ]:
eval_results_before

{'eval_loss': 1.0810511112213135,
 'eval_model_preparation_time': 0.0025,
 'eval_accuracy': 0.46355104749265336,
 'eval_precision_macro': 0.35700305350640954,
 'eval_recall_macro': 0.3409593936150815,
 'eval_f1_macro': 0.2702472377444269,
 'eval_runtime': 642.0055,
 'eval_samples_per_second': 32.863,
 'eval_steps_per_second': 0.514}

In [33]:
# обучение модели
trainer.train()

Epoch,Training Loss,Validation Loss,Accuracy,Precision Macro,Recall Macro,F1 Macro
1,0.723500,0.702876,0.665262,0.658673,0.619927,0.631285
2,0.707900,0.677166,0.682956,0.668319,0.657768,0.662186
3,0.708600,0.674178,0.684852,0.674572,0.655979,0.663387


TrainOutput(global_step=18990, training_loss=0.7384136647911935, metrics={'train_runtime': 14608.7856, 'train_samples_per_second': 31.196, 'train_steps_per_second': 1.3, 'total_flos': 1.1028600761394283e+17, 'train_loss': 0.7384136647911935, 'epoch': 3.0})

In [34]:
# финальные метрики после обучения
eval_results_after = trainer.evaluate()

In [35]:
eval_results_after

{'eval_loss': 0.674177885055542,
 'eval_accuracy': 0.684852155138366,
 'eval_precision_macro': 0.6745719702877455,
 'eval_recall_macro': 0.6559788110036416,
 'eval_f1_macro': 0.6633871945334157,
 'eval_runtime': 976.9891,
 'eval_samples_per_second': 38.874,
 'eval_steps_per_second': 0.811,
 'epoch': 3.0}